# Source Data

![title](Figures/Source.png)

# Single Data Management Solution

The Single Data Management Solution is to generate a master CSV in which each entry of the csv represents the results and metadata of a material specimen. Each material specimen is uniquely identified by it's process (ball milling id / hot press id) and the form at which it was tested (Mixed Powders or Pellet). The major assumption in this data model is that there are no repeat measurements (i.e. single measurement for unique combination of raw material, process condition, and measurement form). If there were repeat measurements then a single measurement could be uniquely identified by an id given at test time or timestamp.

In [1]:
import psycopg2
import pandas as pd
import os
import re

import config
from extract import extract_table_to_df, extract_xlabtxt_to_df, extract_xlabtxts_to_df
from transform import set_df_col_names, flatten_cols, strip_col_whitespace

Configuration variables for my local version of x-materials-database are given in config.py. These will need to be replaced with your own configuration variables.

In [2]:
#Create connection with Postgres Database
connection = psycopg2.connect(host=config.DB_HOST, database=config.DB_NAME, \
                              user=config.DB_USER, password=config.DB_PASS)

# Gather Historical Raw Data and Pipe Into Master CSV

Task Dependencies: <br>
Create Procurement DataFrame >> Create X-Processing DataFrame >> Merge Procurement Office DataFrame with X-Processing DataFrame >> Merge X-Lab DataFrames with Procurement and Processing DataFrame
<< Create XLab DataFrames

### Create Procurement DataFrame

![title](Figures/Procurement.png)

In [3]:
df_material_procurement_cols = ['Cu_uid', 'Se_uid', 'Zn_uid', 'Cu_mass_fraction', \
                               'Se_mass_fraction', 'Zn_mass_fraction']

#pivot table so that ball_milling_uid becomes unique identifier
df_material_procurement = (
                            extract_table_to_df("material_procurement", connection)
                            .pivot(index="ball_milling_uid", columns="material_name")
                            .pipe(flatten_cols)
                            .pipe(set_df_col_names, df_material_procurement_cols)
                          )

In [4]:
df_material_procurement.head()

,Cu_uid,Se_uid,Zn_uid,Cu_mass_fraction,Se_mass_fraction,Zn_mass_fraction
ball_milling_uid,,,,,,
MATX-BM001,MATX-PR-002,MATX-PR-003,MATX-PR-001,0.92,0.06,0.02
MATX-BM002,MATX-PR-005,MATX-PR-006,MATX-PR-004,0.90,0.06,0.04
MATX-BM003,MATX-PR-008,MATX-PR-009,MATX-PR-007,0.88,0.06,0.06
MATX-BM004,MATX-PR-011,MATX-PR-012,MATX-PR-010,0.86,0.06,0.08
MATX-BM005,MATX-PR-014,MATX-PR-015,MATX-PR-013,0.84,0.06,0.10


### Create X-Processing DataFrame

![title](Figures/Processing.png)

In [5]:
df_ball_milling_cols = ['ball_milling_uid', 'ball_milling_process_name', 'milling_time', 'milling_time_units', \
                       'milling_speed', 'milling_speed_units', 'ball_milling_output_material_name', \
                       'ball_milling_output_material_uid', 'hot_press_uid']

df_hot_press_cols = ['hot_press_uid', 'hot_press_process_name', 'hot_press_temperature', 'hot_press_temperature_units', \
               'hot_press_pressure', 'hot_press_pressure_units', 'hot_press_time', 'hot_press_time_units', \
               'hot_press_output_material_name', 'hot_press_output_material_uid']



df_ball_milling = (
                    extract_table_to_df("ball_milling", connection)
                    .pipe(set_df_col_names, df_ball_milling_cols)
                    
                  )

df_hot_press = (
                    extract_table_to_df("hot_press", connection)
                    .pipe(set_df_col_names, df_hot_press_cols)
                    
               )


#Merge df_ball_milling and df_hot_press into df_processing
df_processing = df_ball_milling.merge(df_hot_press, on='hot_press_uid')

In [6]:
df_processing.head()

,ball_milling_uid,ball_milling_process_name,milling_time,milling_time_units,milling_speed,milling_speed_units,ball_milling_output_material_name,ball_milling_output_material_uid,hot_press_uid,hot_press_process_name,hot_press_temperature,hot_press_temperature_units,hot_press_pressure,hot_press_pressure_units,hot_press_time,hot_press_time_units,hot_press_output_material_name,hot_press_output_material_uid
0,MATX-BM001,high energy ball milling,15.0,hr,200.0,rpm,powder,MATX-BM-M001,MATX-HP001,Hot Isostatic Press,850.0,degC,100.0,MPa,1.0,hr,final pellet,MATX-HP-M001
1,MATX-BM002,high energy ball milling,30.0,hr,200.0,rpm,powder,MATX-BM-M002,MATX-HP002,Hot Isostatic Press,850.0,degC,200.0,MPa,0.5,hr,final pellet,MATX-HP-M002
2,MATX-BM003,high energy ball milling,45.0,hr,200.0,rpm,powder,MATX-BM-M003,MATX-HP003,Hot Isostatic Press,900.0,degC,100.0,MPa,1.0,hr,final pellet,MATX-HP-M003
3,MATX-BM004,high energy ball milling,60.0,hr,200.0,rpm,powder,MATX-BM-M004,MATX-HP004,Hot Isostatic Press,900.0,degC,200.0,MPa,0.5,hr,final pellet,MATX-HP-M004
4,MATX-BM005,high energy ball milling,15.0,hr,225.0,rpm,powder,MATX-BM-M005,MATX-HP005,Hot Isostatic Press,1000.0,degC,100.0,MPa,1.0,hr,final pellet,MATX-HP-M005


### Merge Procurement Office DataFrame with X-Processing DataFrame

![title](Figures/ProcurementandProcessing.png)

In [7]:
#Merge X-Processing DataFrame with Procurement Office DataFrame
df_procurement_processing = df_processing.merge(df_material_procurement, on='ball_milling_uid')

In [8]:
df_procurement_processing.head()

,ball_milling_uid,ball_milling_process_name,milling_time,milling_time_units,milling_speed,milling_speed_units,ball_milling_output_material_name,ball_milling_output_material_uid,hot_press_uid,hot_press_process_name,...,hot_press_time,hot_press_time_units,hot_press_output_material_name,hot_press_output_material_uid,Cu_uid,Se_uid,Zn_uid,Cu_mass_fraction,Se_mass_fraction,Zn_mass_fraction
0,MATX-BM001,high energy ball milling,15.0,hr,200.0,rpm,powder,MATX-BM-M001,MATX-HP001,Hot Isostatic Press,...,1.0,hr,final pellet,MATX-HP-M001,MATX-PR-002,MATX-PR-003,MATX-PR-001,0.92,0.06,0.02
1,MATX-BM002,high energy ball milling,30.0,hr,200.0,rpm,powder,MATX-BM-M002,MATX-HP002,Hot Isostatic Press,...,0.5,hr,final pellet,MATX-HP-M002,MATX-PR-005,MATX-PR-006,MATX-PR-004,0.90,0.06,0.04
2,MATX-BM003,high energy ball milling,45.0,hr,200.0,rpm,powder,MATX-BM-M003,MATX-HP003,Hot Isostatic Press,...,1.0,hr,final pellet,MATX-HP-M003,MATX-PR-008,MATX-PR-009,MATX-PR-007,0.88,0.06,0.06
3,MATX-BM004,high energy ball milling,60.0,hr,200.0,rpm,powder,MATX-BM-M004,MATX-HP004,Hot Isostatic Press,...,0.5,hr,final pellet,MATX-HP-M004,MATX-PR-011,MATX-PR-012,MATX-PR-010,0.86,0.06,0.08
4,MATX-BM005,high energy ball milling,15.0,hr,225.0,rpm,powder,MATX-BM-M005,MATX-HP005,Hot Isostatic Press,...,1.0,hr,final pellet,MATX-HP-M005,MATX-PR-014,MATX-PR-015,MATX-PR-013,0.84,0.06,0.10


### Create X-Lab DataFrames

![title](Figures/XLab.png)

In [9]:
df_hall = extract_xlabtxts_to_df("Hall", "x-lab-data")
df_icp = extract_xlabtxts_to_df("ICP", "x-lab-data")

In [10]:
df_hall.head()

material_uid,Measurement,Probe Resistance (ohm),Gas Flow Rate (L/min),Gas Type,Probe Material,Current (mA),Field Strength (T),Sample Position,Magnet Reversal
MATX-BM-M009,Hall,4.3,1,Ar,W,0.1,5,1,True
MATX-HP-M039,Hall,8.2,1,Ar,W,0.1,5,2,True
MATX-BM-M023,Hall,6.1,1,Ar,W,0.1,5,1,True
MATX-BM-M022,Hall,8.6,1,Ar,W,0.1,5,2,True
MATX-HP-M002,Hall,7.5,1,Ar,W,0.1,5,1,True


In [11]:
df_icp.head()

material_uid,Measurement,Pb concentration,Sn concentration,O Concentration,Gas Flow Rate (L/min),Gas Type,Plasma Temperature (celsius),Detector Temperature (celsius),Field Strength (T),Plasma Observation,Radio Frequency (MHz)
MATX-HP-M041,ICP,0.6,0.3,1.2,13,Ar,10000,0,1,Radial,75
MATX-HP-M040,ICP,0.6,0.2,1.0,13,Ar,10000,0,1,Radial,74
MATX-HP-M046,ICP,0.7,0.4,1.0,13,Ar,10000,0,1,Radial,77
MATX-HP-M044,ICP,0.6,0.3,1.0,13,Ar,10000,0,1,Radial,76
MATX-BM-M006,ICP,0.6,0.3,1.1,13,Ar,10000,0,1,Radial,34


### Merge X-Lab DataFrames with Procurement and Processing DataFrame

![title](Figures/XLabProcurementandProcessing.png)

In [12]:
#Split Measurement DataFrames based off of measurement_form
df_measurement_bm = (
                        df_hall[df_hall.index.str.contains('BM')]
                        .join(df_icp[df_icp.index.str.contains('BM')], how='outer')
                        .pipe(strip_col_whitespace)
                        .drop(columns=['Measurement'])
                    )

df_measurement_hp = (
                        df_hall[df_hall.index.str.contains('HP')]
                        .join(df_icp[df_icp.index.str.contains('HP')], how='outer')
                        .pipe(strip_col_whitespace)
                        .drop(columns=['Measurement'])
                    )

#Create separate Procurement_Processing DataFrames for each measurement_form
df_procurement_processing_hp = df_procurement_processing.copy()
df_procurement_processing_bm = df_procurement_processing.copy()

#Create measurement form column
df_procurement_processing_hp['measurement_form'] = 'Pellet'
df_procurement_processing_bm['measurement_form'] = 'Mixed Powders'

#Merge Measurement and Procurement_Processing DataFrames
df_final_bm = pd.merge(df_measurement_bm, df_procurement_processing_bm, how='outer', \
         left_index=True, right_on='ball_milling_output_material_uid')
df_final_hp = pd.merge(df_measurement_hp, df_procurement_processing_hp, how='outer', \
         left_index=True, right_on='hot_press_output_material_uid')

#Concatenate BM and HP DataFrames
df_final = pd.concat([df_final_bm, df_final_hp])

In [13]:
df_final.head()

,Probe Resistance (ohm),Gas Flow Rate (L/min),Gas Type,Probe Material,Current (mA),Field Strength (T),Sample Position,Magnet Reversal,Pb concentration,Sn concentration,...,hot_press_time_units,hot_press_output_material_name,hot_press_output_material_uid,Cu_uid,Se_uid,Zn_uid,Cu_mass_fraction,Se_mass_fraction,Zn_mass_fraction,measurement_form
4,1.5,1,Ar,W,0.1,5,2,True,0.7,0.3,...,hr,final pellet,MATX-HP-M005,MATX-PR-014,MATX-PR-015,MATX-PR-013,0.84,0.06,0.10,Mixed Powders
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6,0.3,...,hr,final pellet,MATX-HP-M006,MATX-PR-017,MATX-PR-018,MATX-PR-016,0.82,0.06,0.12,Mixed Powders
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6,0.3,...,hr,final pellet,MATX-HP-M007,MATX-PR-020,MATX-PR-021,MATX-PR-019,0.80,0.06,0.14,Mixed Powders
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.2,...,hr,final pellet,MATX-HP-M008,MATX-PR-023,MATX-PR-024,MATX-PR-022,0.78,0.06,0.16,Mixed Powders
8,4.3,1,Ar,W,0.1,5,1,True,0.6,0.3,...,hr,final pellet,MATX-HP-M009,MATX-PR-026,MATX-PR-027,MATX-PR-025,0.76,0.06,0.18,Mixed Powders


### Load Historical Data into Master CSV

In [14]:
df_final.to_csv('output_master_csv.csv', index=False)

# Bring In New Data

Hall-MATX-BM-M004.txt, Hall-MATX-HP-M001.txt, and ICP-MATX-BM-M001.txt were taken out of x-lab-data and put into new-data for demonstration purposes.

In [15]:
def add_xlab_txt_file(df_historical, filename):
    """Extract new XLab txt file and combine with historical dataframe."""
    df_new = extract_xlabtxt_to_df(filename)
    mid = df_new.index.values[0] #material id
    if mid.find('BM') != -1:
        df_new = df_new[df_new.index.str.contains('BM')]
        material_id = 'ball_milling_output_material_uid'
        measurement_form = 'Mixed Powders'
    else:
        df_new = df_new[df_new.index.str.contains('HP')]
        material_id = 'hot_press_output_material_uid'
        measurement_form = 'Pellet'
        
    df_new = df_new.pipe(strip_col_whitespace).drop(columns='Measurement')
    #Copy values of new dataframe into the relevant slice of the historical dataframe
    df_historical.loc[(df_historical[material_id] == mid) \
                      & (df_historical['measurement_form'] == measurement_form), df_new.columns] = df_new.values
    
    #print out new values that were added to historical dataframe
    print(df_historical.loc[(df_historical[material_id] == mid) \
                  & (df_historical['measurement_form'] == measurement_form), df_new.columns])
    return df_historical
    

### Extract Historical DataFrame

In [16]:
df_historical = pd.read_csv('master_csv.csv')

### Add New X-Lab Measurement File to Master CSV

In [17]:
df_master = add_xlab_txt_file(df_historical, './new-data/Hall-MATX-BM-M004.txt')

material_uid Probe Resistance (ohm) Gas Flow Rate (L/min) Gas Type  \
37                              3.1                     1       Ar   

material_uid Probe Material Current (mA) Field Strength (T) Sample Position  \
37                        W          0.1                  5               1   

material_uid Magnet Reversal  
37                      True  


In [18]:
df_master = add_xlab_txt_file(df_historical, './new-data/Hall-MATX-HP-M001.txt')

material_uid Probe Resistance (ohm) Gas Flow Rate (L/min) Gas Type  \
81                              2.3                     1       Ar   

material_uid Probe Material Current (mA) Field Strength (T) Sample Position  \
81                        W          0.1                  5               1   

material_uid Magnet Reversal  
81                      True  


In [19]:
df_master = add_xlab_txt_file(df_historical, './new-data/ICP-MATX-BM-M001.txt')

material_uid Pb concentration Sn concentration O Concentration  \
34                        0.5              0.3             1.1   

material_uid Gas Flow Rate (L/min) Gas Type Plasma Temperature (celsius)  \
34                              13       Ar                        10000   

material_uid Detector Temperature (celsius) Field Strength (T)  \
34                                        0                  1   

material_uid Plasma Observation Radio Frequency (MHz)  
34                       Radial                    30  


### Load Master DataFrame into Master CSV

In [20]:
df_master.to_csv('master_csv.csv', index=False)


Materials data can come in many forms, such as technical datasheets (pdf/word documents), raw test data (txt/csv/images/video), metadata (txt/csv/json), test reports (excel/word documents), and materials databases. Materials data is naturally relational, heirarchical, and complex. Yet, most machine learning algorithms require input in the form of an array/matrix. This makes the pipeline to consolidate materials data from many disparate data sources into input for machine learning algorithms very complex. Once the materials data is in array/matrix form it is often very high-dimensional and sparse.